# 7506 - Organizacion de Datos - TP N°2

# 1. Librerias


In [1]:
#Paquetes Clasicos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Paquetes para Preprocesamiento
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
import unicodedata

#Vectorizacion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Pipeline
from sklearn.pipeline import Pipeline

#Tuning de Parametros
from sklearn.model_selection import GridSearchCV

#Clasificadores
from sklearn.linear_model import RidgeClassifier # Classifier using Ridge regression.
from sklearn.linear_model import LogisticRegression # Classifier using Logistic Regression.
from sklearn.linear_model import Perceptron # Classifier using Perceptron.
from sklearn.naive_bayes import MultinomialNB # Classifier using Naive Bayes classifier multinomial
from sklearn.svm import LinearSVC #Classifier using Linear Support Vector
from sklearn.svm import SVC #C-Support Vector
from sklearn.ensemble import GradientBoostingClassifier #Classifier using Gradient Boosting classifier 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2. Datos

In [2]:
#Dataset de entranamiento
df_train = pd.read_csv("train.csv")

In [3]:
#Dataset de test
df_test = pd.read_csv("test.csv")

## 2.1 Dataset 'train'

In [4]:
#Observo la constitucion general del Dataset de entrenamiento

df_train.sample(10)

,id,keyword,location,text,target
5568,7944,rainstorm,"Sicamous, British Columbia",lizzie363 @CstSmith I drove thru deep water in...,1
7349,10521,wildfire,NaN,Reuters Top News: PHOTOS: The Rocky Fire has g...,1
5199,7424,obliterated,"Palmyra, NJ",I was obliterated last night??,0
395,570,arson,"Eldoret, kenya",#Kisii Police in Kisii hunt for students over ...,1
1053,1522,body%20bags,NaN,Womens Handbags Cross Body Geometric Pattern S...,0
2070,2971,dead,NaN,@GailSimone #IWasDisappointedBy TellTale's The...,0
7295,10439,wild%20fires,Twitterville,Collection of Articals and Video of West Coast...,1
5328,7607,pandemonium,NaN,I'll be at SFA very soon....#Pandemonium http:...,1
156,222,airplane%20accident,NaN,Pilot Dies In Plane Crash At Car Festival http...,1
1764,2537,collision,"Peterborough, Ontario, Canada",Two-vehicle collision at Fowlers Corners at Hw...,1


In [5]:
#Dimensiones del dataset

df_train.shape

(7613, 5)

In [6]:
#Cantidad de tweets catalogados como Verdaderos(1) y Falsos(0)

df_train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

**Tratamiento de NaN**

In [7]:
#Reemplazo los NaN por el string 'None'

df_train.location.fillna('None', inplace=True)
df_train.keyword.fillna('None', inplace=True)

In [8]:
#Observo como queda el dataframe

df_train.head()

,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,5,None,None,All residents asked to 'shelter in place' are ...,1
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1


**Tweets Repetidos**

In [9]:
#Cuento la cantidad de 'text' que se encuentran repetidos

duplicados_train = df_train['text'].duplicated(keep=False)
duplicados_train.sum()

179

In [10]:
#Elimino las filas donde 'text' esta repetido

dup_train = df_train[['text','target']][duplicados_train]
dup_train

,text,target
40,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
48,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
106,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0
115,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0
118,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/TH...,0
...,...,...
7600,Evacuation order lifted for town of Roosevelt:...,1
7607,#stormchase Violent Record Breaking EF-5 El Re...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1


In [11]:
#Verifico que el tamaño del dataset disminuyo la cantidad de repetidos

df_train.shape

(7613, 5)

A simple vista se observa que alguno de los tweets repetidos tienen como target diferentes valores, lo cual en principio es confuso. Se decide eliminar todos los tweets repetidos

In [12]:
#Elimino las filas donde 'text' esta repetido

df_train = df_train.drop_duplicates('text',keep=False)

df_train.head()

,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,5,None,None,All residents asked to 'shelter in place' are ...,1
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
#Verifico que el tamaño del dataset disminuyo la cantidad de repetidos

df_train.shape

(7434, 5)

## 2.2 Dataset 'test'

In [14]:
#Observo la constitucion general del Dataset de test

df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [15]:
#Dimensiones del dataset

df_test.shape

(3263, 4)

**Tratamiento de NaN**

In [16]:
#Reemplazo los NaN por el string 'None'

df_test.location.fillna('None', inplace=True)
df_test.keyword.fillna('None', inplace=True)

In [17]:
#Observo como queda el dataframe

df_test.head()

,id,keyword,location,text
0,0,None,None,Just happened a terrible car crash
1,2,None,None,"Heard about #earthquake is different cities, s..."
2,3,None,None,"there is a forest fire at spot pond, geese are..."
3,9,None,None,Apocalypse lighting. #Spokane #wildfires
4,11,None,None,Typhoon Soudelor kills 28 in China and Taiwan


**Tweets Repetidos**

In [18]:
duplicados_test = df_test['text'].duplicated(keep=False)
duplicados_test.sum()

39

In [19]:
dup_test = df_test[['text']][duplicados_test]
dup_test

,text
47,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/TH...
53,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/TH...
128,#AskConnor there's a zombie apocalypse. the it...
285,To fight bioterrorism sir.
286,To fight bioterrorism sir.
510,70 yrs since the atomic bombing of Hiroshima.....
540,.@denisleary Not sure how these folks rush int...
600,.@denisleary Not sure how these folks rush int...
607,Bushfire causes first victim in Albania :: Th...
612,Bushfire causes first victim in Albania :: Th...


En este caso no es necesario eliminar los tweets. Seria conveniente verificar que los mismos fueron catalogados de igual forma al finalizar el analisis

# 3. Preprocesamiento

Se definen funciones especificas para realizar el preprocesamiento de los datos:

In [20]:
# Transformar el texto a minuscula - OK

def minuscula(texto):
    return texto.lower()

In [21]:
#Remover URL - OK

def remover_url(texto):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', texto)

In [22]:
#Remover Usuarios (''@usuario') - OK
#Si aparecen usuarios combinados ('@usuario__usuario') queda '_usuario' y la funcion remover_no_alfabeto(texto) lo elimina

def remover_usuario(texto):
    text = re.sub(r"\@[A-Za-z0-9]+", "", texto)
    return texto

In [23]:
# Remover Emoji - OK

def remover_emoji(texto):
    emoji_patrones = re.compile(
        '['
        u'\U0001F600-\U0001F64F' 
        u'\U0001F300-\U0001F5FF' 
        u'\U0001F680-\U0001F6FF' 
        u'\U0001F1E0-\U0001F1FF' 
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_patrones.sub(r'', texto)

In [24]:
#Expandir las Abreviaturas - OK

abreviaturas = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", 
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

def expandir_abreviatura(texto,mapping = abreviaturas):
    texto = ' '.join([mapping[t] if t in mapping else t for t in texto.split(" ")])
    return texto

In [25]:
#Expandir las Contracciones - OK

contracciones_mapeo = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", 
                       "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                       "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", 
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                       "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                       "mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                       "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
                       "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                       "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                       "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                       "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", 
                       "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", 
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", 
                       "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", 
                       "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", 
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def expandir_contraccion(texto,mapping = contracciones_mapeo):
    specials =["’", "‘", "´", "`"]
    for s in specials:
        texto = texto.replace(s,"'")
    
    texto = ' '.join([mapping[t] if t in mapping else t for t in texto.split(" ")])
    return texto

In [26]:
# Remover los tags HTML - OK

def remover_tag_html(texto):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', texto)

In [27]:
#Remover acentos - OK

def remover_acento(texto):
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return texto

In [28]:
#Remover Puntos - OK

def remover_punto(texto):
    import string
    texto = ''.join([c for c in texto if c not in string.punctuation])
    return texto

In [29]:
#Remover Numeros - OK

def remover_numero(texto):
    texto = ''.join([i for i in texto if not i.isdigit()])
    return texto

In [30]:
#Remover espacios en Blanco (extras/tabs) - OK

def remover_espacio_extra(texto):
    import re
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', texto).strip()

In [31]:
#Remueve todo lo que se sea del alfabeto - OK

def remover_no_alfabeto(texto):
    return ' '.join([i for i in texto.split() if i.isalpha() == True])

In [32]:
#Remover Stop-Word - OK

def remover_stop_word(texto):
    return " ".join ([word for word in word_tokenize(texto) if not word in stopwords.words('english')])

In [33]:
#Lematizar - OK

def lematizar(texto):
    lemma = WordNetLemmatizer()
    return " ".join([lemma.lemmatize(word) for word in word_tokenize(texto)])

Una vez definidias todas las funciones que se utilizaran para realizar la limpieza se define una funcion de ejecuta todas estas a una columna del dataset

In [34]:
def preprocesar_df(df, col_name, clean_col_name):
    df[clean_col_name] = df[col_name].apply(lambda x: minuscula (x))\
                                    .apply(lambda x: remover_url(x))\
                                    .apply(lambda x: remover_usuario(x))\
                                    .apply(lambda x: remover_emoji(x))\
                                    .apply(lambda x: expandir_abreviatura(x))\
                                    .apply(lambda x: expandir_contraccion(x))\
                                    .apply(lambda x: remover_tag_html(x))\
                                    .apply(lambda x: remover_acento(x))\
                                    .apply(lambda x: remover_punto(x))\
                                    .apply(lambda x: remover_numero(x))\
                                    .apply(lambda x: remover_espacio_extra(x))\
                                    .apply(lambda x: remover_no_alfabeto(x))\
                                    .apply(lambda x: remover_stop_word(x))\
                                    .apply(lambda x: lematizar(x))

## 3.1 Preprocesamiento del Dataset 'train' 

Realizo el preprocesamiento de la comlumna **'text'** del dataset **'train'**

In [35]:
#Llamo a la funcion que realiza el preprocesamiento

preprocesar_df(df_train,'text', 'texto_preprocesado')

In [36]:
#Observo las 5 primeras filas

df_train.head(30)

,id,keyword,location,text,target,texto_preprocesado
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,4,None,None,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,None,None,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...
5,8,None,None,#RockyFire Update => California Hwy. 20 closed...,1,rockyfire update california hwy closed directi...
6,10,None,None,#flood #disaster Heavy rain causes flash flood...,1,flood disaster heavy rain cause flash flooding...
7,13,None,None,I'm on top of the hill and I can see a fire in...,1,top hill see fire wood
8,14,None,None,There's an emergency evacuation happening now ...,1,emergency evacuation happening building across...
9,15,None,None,I'm afraid that the tornado is coming to our a...,1,afraid tornado coming area


In [37]:
#Dimensiones del dataset

df_train.shape

(7434, 6)

Defino los vectores que se utilizaran para el entrenamiento de los modelos

In [38]:
x_train_original = df_train['text']

In [39]:
x_train_preprocesado = df_train['texto_preprocesado']

In [40]:
y_train = df_train['target']

## 3.2 Preprocesamiento del Dataset 'test' 

Realizo el preprocesamiento de la comlumna **'text'** del dataset **'test'**

In [41]:
preprocesar_df(df_test,'text', 'texto_preprocesado')

In [42]:
#Observo las 5 primeras filas

df_test.head()

,id,keyword,location,text,texto_preprocesado
0,0,None,None,Just happened a terrible car crash,happened terrible car crash
1,2,None,None,"Heard about #earthquake is different cities, s...",heard earthquake different city stay safe ever...
2,3,None,None,"there is a forest fire at spot pond, geese are...",forest fire spot pond goose fleeing across str...
3,9,None,None,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfire
4,11,None,None,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill china taiwan


In [43]:
df_test.shape

(3263, 5)

Defino los vectores que se utilizaran para el entrenamiento de los modelos

In [44]:
x_test_original = df_test['text']

In [45]:
x_test_preprocesado = df_test['texto_preprocesado']

# 4. Hiperparametros de Vectorizadores

Se definen los Hiperparametros que se utilizaran en GridSearchCV para realizar el tuning

In [51]:
#Parametros para el vectorizador BOW - CountVectorizer()

parametros_bow = {'vectorizador__strip_accents':'unicode',
                  'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                  'vectorizador__max_df':[10,20,30],
                  'vectorizador__binary':[False,True]}

In [52]:
#Parametros para el vectorizador TFIDF - TfidfVectorizer()

parametros_tfidf = {'vectorizador__strip_accents':'unicode',
                  'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                  'vectorizador__max_df':[10,20,30],
                  'vectorizador__binary':[False,True],
                  'vectorizador__use_idf':[True,False]}

# 5. Tuning de Parametros (GridSearch) y Resultados

## 5.1 Ridge Regresion

### 5.1.1 BOW

In [91]:
#Pipeline

pipeline_bow_RR = Pipeline([('vectorizador', CountVectorizer()),('clf', RidgeClassifier())])

**Datos sin Preprocesar**

In [104]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_1 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__min_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'clf__alpha': [0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__normalize':[False,True],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_RR_bow_original = GridSearchCV(pipeline_bow_RR, hiperparametros_1,cv=5, n_jobs=-1,verbose=2)
clf_RR_bow_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_RR_bow_original.best_score_)

print("Mejores Parametros: ", clf_RR_bow_original.best_params_)

Fitting 5 folds for each of 468 candidates, totalling 2340 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 2340 out of 2340 | elapsed:  7.6min finished


Mejor Score:  0.7152371246179314
Mejores Parametros:  {'clf__alpha': 2, 'clf__normalize': True, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 10, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [105]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [106]:
submission['target'] = clf_RR_bow_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [107]:
submission.to_csv("original_bow_hiper3_ridge_regression.csv", index=False)

**Datos Preprocesados**

In [108]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_2 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__min_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'clf__alpha': [0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__normalize':[False,True],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_RR_bow_preprocesado = GridSearchCV(pipeline_bow_RR, hiperparametros_2,cv=5, n_jobs=-1,verbose=2)
clf_RR_bow_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_RR_bow_preprocesado.best_score_)

print("Mejores Parametros: ", clf_RR_bow_preprocesado.best_params_)

Fitting 5 folds for each of 468 candidates, totalling 2340 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   53.2s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2340 out of 2340 | elapsed:  4.8min finished


Mejor Score:  0.6830867971047417
Mejores Parametros:  {'clf__alpha': 2, 'clf__normalize': True, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 10, 'vectorizador__ngram_range': (1, 1), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [109]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [110]:
submission['target'] = clf_RR_bow_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [111]:
submission.to_csv("preprocesado_bow_hiper3_ridge_regression.csv", index=False)

### 5.1.2 TFIDF 

In [112]:
#Pipeline

pipeline_tfidf_RR = Pipeline([('vectorizador', TfidfVectorizer()),('clf', RidgeClassifier())])

**Datos sin Preprocesar**

In [114]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_3 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__max_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'vectorizador__use_idf':[True,False],
                     'clf__alpha': [0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__normalize':[False,True],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_RR_tfidf_original = GridSearchCV(pipeline_tfidf_RR, hiperparametros_3,cv=5, n_jobs=-1,verbose=2)
clf_RR_tfidf_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_RR_tfidf_original.best_score_)

print("Mejores Parametros: ", clf_RR_tfidf_original.best_params_)

Fitting 5 folds for each of 936 candidates, totalling 4680 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   58.7s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 4680 out of 4680 | elapsed: 19.3min finished


Mejor Score:  0.7097176878845011
Mejores Parametros:  {'clf__alpha': 4, 'clf__normalize': True, 'clf__random_state': 42, 'vectorizador__binary': False, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [115]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [116]:
submission['target'] = clf_RR_tfidf_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [117]:
submission.to_csv("original_tfidf_hiper3_ridge_regression.csv", index=False)

**Datos Preprocesados**

In [118]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_4 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__max_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'vectorizador__use_idf':[True,False],
                     'clf__alpha': [0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__normalize':[False,True],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_RR_tfidf_preprocesado = GridSearchCV(pipeline_tfidf_RR, hiperparametros_4,cv=5, n_jobs=-1,verbose=2)
clf_RR_tfidf_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_RR_tfidf_preprocesado.best_score_)

print("Mejores Parametros: ", clf_RR_tfidf_preprocesado.best_params_)

Fitting 5 folds for each of 936 candidates, totalling 4680 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 4680 out of 4680 | elapsed: 11.8min finished


Mejor Score:  0.6997636764023059
Mejores Parametros:  {'clf__alpha': 3, 'clf__normalize': True, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': False}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [119]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [120]:
submission['target'] = clf_RR_tfidf_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [121]:
submission.to_csv("preprocesado_tfidf_hiper3_ridge_regression.csv", index=False)

## 5.2 Logistic Regression

### 5.2.1 BOW

In [122]:
#Pipeline

pipeline_bow_LR = Pipeline([('vectorizador', CountVectorizer()),('clf', LogisticRegression())])

**Datos sin Preprocesar**

In [124]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_5 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__min_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'clf__C': [0.0001,0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__penalty': ['l1', 'l2'],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_LR_bow_original = GridSearchCV(pipeline_bow_LR, hiperparametros_5,cv=5, n_jobs=-1,verbose=2)
clf_LR_bow_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_LR_bow_original.best_score_)

print("Mejores Parametros: ", clf_LR_bow_original.best_params_)

Fitting 5 folds for each of 504 candidates, totalling 2520 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 2520 out of 2520 | elapsed:  7.2min finished


Mejor Score:  0.7276094931306857
Mejores Parametros:  {'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 10, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [126]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [127]:
submission['target'] = clf_LR_bow_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [128]:
submission.to_csv("original_bow_hiper3_logistic_regression.csv", index=False)

**Datos Preprocesados**

In [129]:
#Dataset Preprocesados

#Hiperparametros del vectorizador y del clasificador
hiperparametros_6 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__min_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'clf__C': [0.0001,0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__penalty': ['l1', 'l2'],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_LR_bow_preprocesado = GridSearchCV(pipeline_bow_LR, hiperparametros_6,cv=5, n_jobs=-1,verbose=2)
clf_LR_bow_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_LR_bow_preprocesado.best_score_)

print("Mejores Parametros: ", clf_LR_bow_preprocesado.best_params_)

Fitting 5 folds for each of 504 candidates, totalling 2520 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 2520 out of 2520 | elapsed:  4.5min finished


Mejor Score:  0.693846716405347
Mejores Parametros:  {'clf__C': 0.5, 'clf__penalty': 'l2', 'clf__random_state': 42, 'vectorizador__binary': False, 'vectorizador__min_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [130]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [131]:
submission['target'] = clf_LR_bow_preprocesado.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [132]:
submission.to_csv("preprocesado_bow_hiper3_logistic_regression.csv", index=False)

### 5.2.2 TFIDF 

In [133]:
#Pipeline

pipeline_tfidf_LR = Pipeline([('vectorizador', TfidfVectorizer()),('clf', LogisticRegression())])

**Datos sin Preprocesar**

In [134]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_7 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__max_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'vectorizador__use_idf':[True,False],
                     'clf__C': [0.0001,0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__penalty': ['l1', 'l2'],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_LR_tfidf_original = GridSearchCV(pipeline_tfidf_LR, hiperparametros_7,cv=5, n_jobs=-1,verbose=2)
clf_LR_tfidf_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_LR_tfidf_original.best_score_)

print("Mejores Parametros: ", clf_LR_tfidf_original.best_params_)

Fitting 5 folds for each of 1008 candidates, totalling 5040 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed: 25.7min
[Parallel(n_jobs=-1)]: Done 5040 out of 5040 | elapsed: 27.4min finished


Mejor Score:  0.7093148244860573
Mejores Parametros:  {'clf__C': 5, 'clf__penalty': 'l2', 'clf__random_state': 42, 'vectorizador__binary': False, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [135]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [136]:
submission['target'] = clf_LR_tfidf_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [137]:
submission.to_csv("original_tfidf_hiper3_logistic_regression.csv", index=False)

**Datos Preprocesados**

In [138]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_8 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__max_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'vectorizador__use_idf':[True,False],
                     'clf__C': [0.0001,0.001,0.01,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5],
                     'clf__penalty': ['l1', 'l2'],
                     'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_LR_tfidf_preprocesado = GridSearchCV(pipeline_tfidf_LR, hiperparametros_8,cv=5, n_jobs=-1,verbose=2)
clf_LR_tfidf_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_LR_tfidf_preprocesado.best_score_)

print("Mejores Parametros: ", clf_LR_tfidf_preprocesado.best_params_)

Fitting 5 folds for each of 1008 candidates, totalling 5040 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   56.4s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed: 16.8min
[Parallel(n_jobs=-1)]: Done 5040 out of 5040 | elapsed: 17.9min finished


Mejor Score:  0.6880613590552849
Mejores Parametros:  {'clf__C': 2, 'clf__penalty': 'l2', 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [139]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [140]:
submission['target'] = clf_LR_tfidf_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [141]:
submission.to_csv("preprocesado_tfidf_hiper3_logistic_regression.csv", index=False)

## 5.3 Naive Bayes (Multinomial)

### 5.3.1 BOW

In [142]:
#Pipeline

pipeline_bow_NB = Pipeline([('vectorizador', CountVectorizer()),('clf', MultinomialNB())])

**Datos sin Preprocesar**

In [149]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_9 = {'vectorizador__strip_accents':['unicode'],
                     'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                     'vectorizador__min_df':[10,20,30],
                     'vectorizador__binary':[False,True],
                     'clf__alpha': [0.0001,0.001,0.01,0.05,0.1,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10]}

#Tuning de Hiperparametros
clf_NB_bow_original = GridSearchCV(pipeline_bow_NB, hiperparametros_9,cv=5, n_jobs=-1,verbose=2)
clf_NB_bow_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_NB_bow_original.best_score_)

print("Mejores Parametros: ", clf_NB_bow_original.best_params_)

Fitting 5 folds for each of 342 candidates, totalling 1710 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1710 out of 1710 | elapsed:  4.6min finished


Mejor Score:  0.7184628376390811
Mejores Parametros:  {'clf__alpha': 6, 'vectorizador__binary': False, 'vectorizador__min_df': 10, 'vectorizador__ngram_range': (1, 1), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [150]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [151]:
submission['target'] = clf_NB_bow_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [152]:
submission.to_csv("original_bow_hiper3_naive_bayes.csv", index=False)

**Datos Preprocesados**

In [153]:
#Dataset Preprocesados

#Hiperparametros del vectorizador y del clasificador
hiperparametros_10 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__min_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'clf__alpha': [0.0001,0.001,0.01,0.05,0.1,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10]}

#Tuning de Hiperparametros
clf_NB_bow_preprocesado = GridSearchCV(pipeline_bow_NB, hiperparametros_10,cv=5, n_jobs=-1,verbose=2)
clf_NB_bow_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_NB_bow_preprocesado.best_score_)

print("Mejores Parametros: ", clf_NB_bow_preprocesado.best_params_)

Fitting 5 folds for each of 342 candidates, totalling 1710 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1710 out of 1710 | elapsed:  3.2min finished


Mejor Score:  0.7029953631336998
Mejores Parametros:  {'clf__alpha': 9, 'vectorizador__binary': False, 'vectorizador__min_df': 10, 'vectorizador__ngram_range': (1, 1), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [170]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [171]:
submission['target'] = clf_NB_bow_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [172]:
submission.to_csv("preprocesado_bow_hiper3_naive_bayes.csv", index=False)

### 5.3.2 TFIDF 

In [158]:
#Pipeline

pipeline_tfidf_NB = Pipeline([('vectorizador', TfidfVectorizer()),('clf', MultinomialNB())])

**Datos sin Preprocesar**

In [159]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_11 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__alpha': [0.0001,0.001,0.01,0.05,0.1,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10]}

#Tuning de Hiperparametros
clf_NB_tfidf_original = GridSearchCV(pipeline_tfidf_NB, hiperparametros_11,cv=5, n_jobs=-1,verbose=2)
clf_NB_tfidf_original.fit(x_train_original, y_train)



#Muestro los resultados

print("Mejor Score: ", clf_NB_tfidf_original.best_score_)

print("Mejores Parametros: ", clf_NB_tfidf_original.best_params_)

Fitting 5 folds for each of 684 candidates, totalling 3420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 3420 out of 3420 | elapsed: 12.9min finished


Mejor Score:  0.7199424170536755
Mejores Parametros:  {'clf__alpha': 1, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [160]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [161]:
submission['target'] = clf_NB_tfidf_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [162]:
submission.to_csv("original_tfidf_hiper3_naive_bayes.csv", index=False)

**Datos Preprocesados**

In [163]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_12 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__alpha': [0.0001,0.001,0.01,0.05,0.1,1,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9,10]}

#Tuning de Hiperparametros
clf_NB_tfidf_preprocesado = GridSearchCV(pipeline_tfidf_NB, hiperparametros_12,cv=5, n_jobs=-1,verbose=2)
clf_NB_tfidf_preprocesado.fit(x_train_preprocesado, y_train)



#Muestro los resultados

print("Mejor Score: ", clf_NB_tfidf_preprocesado.best_score_)

print("Mejores Parametros: ", clf_NB_tfidf_preprocesado.best_params_)

Fitting 5 folds for each of 684 candidates, totalling 3420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 3420 out of 3420 | elapsed:  7.8min finished


Mejor Score:  0.701512434820938
Mejores Parametros:  {'clf__alpha': 1, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [164]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [165]:
submission['target'] = clf_NB_tfidf_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [166]:
submission.to_csv("preprocesado_tfidf_hiper3_naive_bayes.csv", index=False)

## 5.4 Gradient Boosting

### 5.4.1 BOW

In [46]:
#Pipeline

pipeline_bow_GB = Pipeline([('vectorizador', CountVectorizer()),('clf', GradientBoostingClassifier())])

**Datos sin Preprocesar**

In [177]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_13 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__min_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'clf__learning_rate': [0.075,0.1,0.125,0.15],
                      'clf__n_estimators': [100,200,300],
                      'clf__max_depth': [3,4,5,6],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_GB_bow_original = GridSearchCV(pipeline_bow_GB, hiperparametros_13,cv=5, n_jobs=-1,verbose=2)
clf_GB_bow_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_GB_bow_original.best_score_)

print("Mejores Parametros: ", clf_GB_bow_original.best_params_)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 36.0min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 46.5min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 56.0min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed: 61.6min finished


Mejor Score:  0.7180561727886637
Mejores Parametros:  {'clf__learning_rate': 0.075, 'clf__max_depth': 6, 'clf__n_estimators': 300, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 30, 'vectorizador__ngram_range': (1, 1), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [178]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [179]:
submission['target'] = clf_GB_bow_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [180]:
submission.to_csv("original_bow_hiper3_gradient_boosting.csv", index=False)

**Datos Preprocesados**

In [181]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_14 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__min_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'clf__learning_rate': [0.075,0.1,0.125,0.15],
                      'clf__n_estimators': [100,200,300],
                      'clf__max_depth': [3,4,5,6],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_GB_bow_preprocesado = GridSearchCV(pipeline_bow_GB, hiperparametros_14,cv=5, n_jobs=-1,verbose=2)
clf_GB_bow_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_GB_bow_preprocesado.best_score_)

print("Mejores Parametros: ", clf_GB_bow_preprocesado.best_params_)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 28.2min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 34.6min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed: 37.7min finished


Mejor Score:  0.6903503762079792
Mejores Parametros:  {'clf__learning_rate': 0.15, 'clf__max_depth': 6, 'clf__n_estimators': 300, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [182]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [183]:
submission['target'] = clf_GB_bow_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [184]:
submission.to_csv("preprocesado_bow_hiper3_gradient_boosting.csv", index=False)

### 5.4.2 TFIDF

In [47]:
#Pipeline

pipeline_tfidf_GB = Pipeline([('vectorizador', TfidfVectorizer()),('clf', GradientBoostingClassifier())])

**Datos sin Preprocesar**

In [49]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_15 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__learning_rate': [0.1,0.125,0.15],
                      'clf__n_estimators': [100,200,300],
                      'clf__max_depth': [3,4,5,6],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_GB_tfidf_original = GridSearchCV(pipeline_tfidf_GB, hiperparametros_15,cv=5, n_jobs=-1,verbose=2)
clf_GB_tfidf_original.fit(x_train_original, y_train)



#Muestro los resultados

print("Mejor Score: ", clf_GB_tfidf_original.best_score_)

print("Mejores Parametros: ", clf_GB_tfidf_original.best_params_)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 96.9min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 158.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 209.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 273.8min


KeyboardInterrupt: 

Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [160]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [161]:
submission['target'] = clf_GB_tfidf_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [162]:
submission.to_csv("original_tfidf_hiper3_gradient_boosting.csv", index=False)

**Datos Preprocesados**

In [163]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_16 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__learning_rate': [0.1,0.125,0.15],
                      'clf__n_estimators': [100,200,300],
                      'clf__max_depth': [3,4,5,6],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_GB_tfidf_preprocesado = GridSearchCV(pipeline_tfidf_GB, hiperparametros_16,cv=5, n_jobs=-1,verbose=2)
clf_GB_tfidf_preprocesado.fit(x_train_preprocesado, y_train)



#Muestro los resultados

print("Mejor Score: ", clf_GB_tfidf_preprocesado.best_score_)

print("Mejores Parametros: ", clf_GB_tfidf_preprocesado.best_params_)

Fitting 5 folds for each of 684 candidates, totalling 3420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 3420 out of 3420 | elapsed:  7.8min finished


Mejor Score:  0.701512434820938
Mejores Parametros:  {'clf__alpha': 1, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [164]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [165]:
submission['target'] = clf_GB_tfidf_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [166]:
submission.to_csv("preprocesado_tfidf_hiper3_gradient_boosting.csv", index=False)

##  5.5 Linear Support Vector

### 5.5.1 BOW

In [49]:
#Pipeline

pipeline_bow_SV = Pipeline([('vectorizador', CountVectorizer()),('clf', LinearSVC())])

**Datos sin Preprocesar**

In [52]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_25 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__min_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'clf__C': [0.001,0.01,0.25,0.5,0.75,1],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SV_bow_original = GridSearchCV(pipeline_bow_SV, hiperparametros_25,cv=5, n_jobs=-1,verbose=2)
clf_SV_bow_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_SV_bow_original.best_score_)

print("Mejores Parametros: ", clf_SV_bow_original.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  2.3min finished


Mejor Score:  0.7273414907665445
Mejores Parametros:  {'clf__C': 0.01, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 10, 'vectorizador__ngram_range': (1, 1), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [53]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [54]:
submission['target'] = clf_SV_bow_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [55]:
submission.to_csv("original_bow_hiper3_linear_support_vector.csv", index=False)

**Datos Preprocesados**

In [56]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_26 = {'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__min_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'clf__C': [0.001,0.01,0.25,0.5,0.75,1],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SV_bow_preprocesado = GridSearchCV(pipeline_bow_SV, hiperparametros_26,cv=5, n_jobs=-1,verbose=2)
clf_SV_bow_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_SV_bow_preprocesado.best_score_)

print("Mejores Parametros: ", clf_SV_bow_preprocesado.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   49.8s


Mejor Score:  0.689273479170306
Mejores Parametros:  {'clf__C': 0.01, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 10, 'vectorizador__ngram_range': (1, 1)}


[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  1.4min finished


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [57]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [58]:
submission['target'] = clf_SV_bow_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [59]:
submission.to_csv("preprocesado_bow_hiper3_linear_support_vector.csv", index=False)

### 5.5.2 TFIDF

In [61]:
#Pipeline

pipeline_tfidf_SV = Pipeline([('vectorizador', TfidfVectorizer()),('clf', LinearSVC())])

**Datos sin Preprocesar**

In [62]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_27 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__C': [0.001,0.01,0.25,0.5,0.75,1],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SV_tfidf_original = GridSearchCV(pipeline_tfidf_SV,hiperparametros_27,cv=5, n_jobs=-1,verbose=2)
clf_SV_tfidf_original.fit(x_train_original, y_train)



#Muestro los resultados

print("Mejor Score: ", clf_SV_tfidf_original.best_score_)

print("Mejores Parametros: ", clf_SV_tfidf_original.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  6.0min finished


Mejor Score:  0.7066244826178609
Mejores Parametros:  {'clf__C': 0.25, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [63]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [64]:
submission['target'] = clf_SV_tfidf_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [65]:
submission.to_csv("original_tfidf_hiper3_linear_support_vector.csv", index=False)

**Datos Preprocesados**

In [66]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_28 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__C': [0.001,0.01,0.25,0.5,0.75,1],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SV_tfidf_preprocesado = GridSearchCV(pipeline_tfidf_SV,hiperparametros_28,cv=5, n_jobs=-1,verbose=2)
clf_SV_tfidf_preprocesado.fit(x_train_preprocesado, y_train)



#Muestro los resultados

print("Mejor Score: ", clf_SV_tfidf_preprocesado.best_score_)

print("Mejores Parametros: ", clf_SV_tfidf_preprocesado.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  3.6min finished


Mejor Score:  0.6891368079207777
Mejores Parametros:  {'clf__C': 0.25, 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [67]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [68]:
submission['target'] = clf_SV_tfidf_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [69]:
submission.to_csv("preprocesado_tfidf_hiper3_linear_support_vector.csv", index=False)

##  5.6 C-Support Vector

### 5.6.1 BOW

In [71]:
#Pipeline

pipeline_bow_SVC = Pipeline([('vectorizador', CountVectorizer()),('clf', SVC())])

**Datos sin Preprocesar**

In [72]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_29 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__min_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'clf__C': [0.001,0.01,0.25,0.5,0.75,1],
                      'clf__kernel':['poly', 'rbf', 'sigmoid', 'precomputed'],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SVC_bow_original = GridSearchCV(pipeline_bow_SVC, hiperparametros_29,cv=5, n_jobs=-1,verbose=2)
clf_SVC_bow_original.fit(x_train_original, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_SVC_bow_original.best_score_)

print("Mejores Parametros: ", clf_SVC_bow_original.best_params_)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 29.1min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 38.4min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 54.3min
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed: 57.6min finished


Mejor Score:  0.7284142243091993
Mejores Parametros:  {'clf__C': 0.25, 'clf__kernel': 'sigmoid', 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__min_df': 20, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [73]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [74]:
submission['target'] = clf_SVC_bow_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [75]:
submission.to_csv("original_bow_hiper3_support_vector.csv", index=False)

**Datos Preprocesados**

In [76]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_30 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__min_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'clf__C': [0.001,0.01,0.25,0.5,0.75,1],
                      'clf__kernel':['poly', 'rbf', 'sigmoid', 'precomputed'],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SVC_bow_preprocesado = GridSearchCV(pipeline_bow_SVC, hiperparametros_30,cv=5, n_jobs=-1,verbose=2)
clf_SVC_bow_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_SVC_bow_preprocesado.best_score_)

print("Mejores Parametros: ", clf_SVC_bow_preprocesado.best_params_)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   46.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 29.3min
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed: 30.6min finished


Mejor Score:  0.6900815592469867
Mejores Parametros:  {'clf__C': 0.75, 'clf__kernel': 'rbf', 'clf__random_state': 42, 'vectorizador__binary': False, 'vectorizador__min_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode'}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [77]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [78]:
submission['target'] = clf_SV_bow_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [79]:
submission.to_csv("preprocesado_bow_hiper3_support_vector.csv", index=False)

### 5.6.2 TFIDF

In [49]:
#Pipeline

pipeline_tfidf_SVC = Pipeline([('vectorizador', TfidfVectorizer()),('clf', SVC())])

**Datos sin Preprocesar**

In [50]:
#Dataset sin preprocesamiento

#Hiperparametros del vectorizador y del clasificador
hiperparametros_31 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__C': [1],
                      'clf__kernel':['poly', 'rbf', 'sigmoid', 'precomputed'],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SVC_tfidf_original = GridSearchCV(pipeline_tfidf_SVC,hiperparametros_31,cv=5, n_jobs=-1,verbose=2)
clf_SVC_tfidf_original.fit(x_train_original, y_train)



#Muestro los resultados

print("Mejor Score: ", clf_SVC_tfidf_original.best_score_)

print("Mejores Parametros: ", clf_SVC_tfidf_original.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 26.0min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 26.3min finished


Mejor Score:  0.7046053685552944
Mejores Parametros:  {'clf__C': 1, 'clf__kernel': 'sigmoid', 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 2), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [51]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [52]:
submission['target'] = clf_SVC_tfidf_original.predict(x_test_original)

Guardo el archivo .csv para realizar el submit a Kaggle

In [53]:
submission.to_csv("original_tfidf_hiper3_support_vector.csv", index=False)

**Datos Preprocesados**

In [55]:
#Dataset Preprocesado

#Hiperparametros del vectorizador y del clasificador
hiperparametros_32 = {'vectorizador__strip_accents':['unicode'],
                      'vectorizador__ngram_range': [(1,1),(2,2),(1,2)],
                      'vectorizador__max_df':[10,20,30],
                      'vectorizador__binary':[False,True],
                      'vectorizador__use_idf':[True,False],
                      'clf__C': [1],
                      'clf__kernel':['poly', 'rbf', 'sigmoid', 'precomputed'],
                      'clf__random_state':[42]}

#Tuning de Hiperparametros
clf_SVC_tfidf_preprocesado = GridSearchCV(pipeline_tfidf_SVC,hiperparametros_32,cv=5, n_jobs=-1,verbose=2)
clf_SVC_tfidf_preprocesado.fit(x_train_preprocesado, y_train)


#Muestro los resultados

print("Mejor Score: ", clf_SVC_tfidf_preprocesado.best_score_)

print("Mejores Parametros: ", clf_SVC_tfidf_preprocesado.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 19.2min finished


Mejor Score:  0.6833514505707156
Mejores Parametros:  {'clf__C': 1, 'clf__kernel': 'rbf', 'clf__random_state': 42, 'vectorizador__binary': True, 'vectorizador__max_df': 30, 'vectorizador__ngram_range': (1, 1), 'vectorizador__strip_accents': 'unicode', 'vectorizador__use_idf': True}


Generacion del SUBMIT

Leo el archivo .csv modelo que tenemos se utilizar para realizar el submit a Kaggle

In [56]:
submission = pd.read_csv('sample_submission.csv')

Creo una nueva columna con los valores que predice el modelo

In [57]:
submission['target'] = clf_SVC_tfidf_preprocesado.predict(x_test_preprocesado)

Guardo el archivo .csv para realizar el submit a Kaggle

In [58]:
submission.to_csv("preprocesado_tfidf_hiper3_support_vector.csv", index=False)